In [58]:
import warnings
warnings.filterwarnings('ignore')

### 목표
- 네이버 영화리뷰 데이터셋을 이용해 긍정리뷰 / 부정리뷰를 분류하는 모델을 만들어보자
- TF-IDF 방법을 적용해보자
- Konlpy 한국어 형태소 분석기를 설치하거 활용해보자
- 단어별 긍정 / 부정 정보를 시각화해서 확인해보자

In [59]:
# Java 버전 확인하기
# 자바 버전은 cmd창에서 java -version으로 확인 가능
# 1.7버전 이상 설치 필요

In [60]:
# jpype1 : java 라이브러리인 konlpy를 파이썬 환경에서 구동되도록 도와주는 라이브러리
%pip install jpype1

Note: you may need to restart the kernel to use updated packages.


In [61]:
# konlpy 설치
# konlpy -> 한국어 형태소 분석기 모음집
%pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [62]:
from konlpy.tag import Kkma

In [63]:
kkma = Kkma()

In [64]:
kkma.nouns('나비')

['나비']

### os에러가 뜨는 경우
- jvm.dll 파일을 찾을 수 없다는 에러가 뜸
- 시스템 환경 변수가 제대로 잡혔는지 확인을 해보기

In [65]:
import os # 시스템 접근

In [66]:
print('JAVA_HOME' in os.environ)

True


In [67]:
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk1.8.0_202\bin'

In [68]:
print('JAVA_HOME' in os.environ)

True


### 네이버 데이터 로딩

In [69]:
import pandas as pd

In [70]:
df_train = pd.read_csv('./data/ratings_train.csv')
df_test = pd.read_csv('./data/ratings_test.csv')

In [71]:
df_train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [72]:
# id : 평점과 리뷰를 적은 사용자의 번호
# document : 실제 리뷰 내용
# label : 긍정 (1) / 부정 (0)

In [73]:
df_train.info(), df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


(None, None)

In [74]:
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [75]:
df_train.info(), df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


(None, None)

### 데이터 분할
- 문제와 정답 데이터로 분할

In [76]:
text_train = df_train['document']
y_train = df_train['label']
text_test = df_test['document']
y_test = df_test['label']

In [77]:
print(text_train.shape)
print(y_train.shape)
print(text_test.shape)
print(y_test.shape)

(149995,)
(149995,)
(49997,)
(49997,)


** Vectorizer **
- Vectorizer라는 도구는 말 그대로 대상을 벡터라는 수치값으로 변환해주는 도구
- 기계학습 분야에서는 주로 텍스트데이터를 쉽게 분석하기 위해 벡터화 시켜주는 경우가 많다
- 숫자나 수치형의 벡터를 입력받기를 기대하는 머신러닝 모델을 실행하기 위해서  
  텍스트나 다른 형태의 데이터를 숫자나 벡터로 나타낼 필요가 있다

### BOW(단어 가방, Bag Of Word)
- 문서 또는 말뭉치를 벡터라는 수치값으로 변환하는 기본적인 방법
- 문서 내에 있는 모든 단어를 모아 하나의 가방으로 만들어서 특정 문서에 어떤 단어가 들어있는지  
  리스트 형태로 표현하고 순서-숫자 형태로 나타내는 방법

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [79]:
text_train[:3]

0                  아 더빙.. 진짜 짜증나네요 목소리
1    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                    너무재밓었다그래서보는것을추천한다
Name: document, dtype: object

In [80]:
tf_idf_vect = TfidfVectorizer()

# 단어 가방 만들기
tf_idf_vect.fit(text_train[:3])

# 분리된 문장으로 만든 단어사전 출력
tf_idf_vect.vocabulary_.keys()

dict_keys(['더빙', '진짜', '짜증나네요', '목소리', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나', '너무재밓었다그래서보는것을추천한다'])

### 한국어 형태소 분석기
** konlpy-Kkma ** 꼬꼬미, 서울대학교 IDS(Intelligent Data System) 연구실 개발
- nouns() : 명사
- morphs() : 형태소 추출
- pos() : 형태소 추출 + 품사 부착

In [81]:
from konlpy.tag import Kkma

In [82]:
kkma = Kkma()

In [83]:
text_train[0]

'아 더빙.. 진짜 짜증나네요 목소리'

In [84]:
kkma.nouns(text_train[0])

['더빙', '목소리']

In [85]:
# TF-IDF와 kkma.nouns() 연결해서 한국어 명사 추출기 만들기
def myTokenizer(text):
    return kkma.nouns(text)

In [86]:
tmp_tf_idf = TfidfVectorizer(tokenizer=myTokenizer)
tmp_tf_idf.fit(text_train[:3])
tmp_tf_idf.vocabulary_.keys()

dict_keys(['더빙', '목소리', '흠', '포스터', '포스터보고', '보고', '초', '초딩영화줄', '딩', '영화', '줄', '오버', '오버연기', '연기', '재', '재밓', '밓', '추천'])

### pos tagging 활용
- 형태소를 추출해서 품사를 태킹해주는 기능

In [87]:
kkma.tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [88]:
# 단순 형태소만 추출
data = '집 가다 가고싶다 배고프다 먹다 모자 안경'
kkma.morphs(data)

['집', '가다', '가', '고', '싶', '다', '배고프', '다', '먹', '다', '모자', '안경']

In [89]:
kkma.pos(data)

[('집', 'NNG'),
 ('가다', 'MAG'),
 ('가', 'VV'),
 ('고', 'ECE'),
 ('싶', 'VXA'),
 ('다', 'ECS'),
 ('배고프', 'VA'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('다', 'ECS'),
 ('모자', 'NNG'),
 ('안경', 'NNG')]

In [90]:
d = pd.DataFrame(kkma.pos(data), columns=['morphs', 'tag'])
d.set_index('tag', inplace=True)

# 명사, 형용사, 동사만 추출해보기
d.loc[d.index.intersection(['VV','VA','NNG'])]

,morphs
tag,
NNG,집
NNG,모자
NNG,안경
VV,가
VV,먹
VA,배고프
